![임도형 커멘트](https://github.com/dhrim/keras_example_seminia_2020/raw/master/comment.png)

# 개요

- 원 본 : https://keras.io/examples/nlp/addition_rnn/
- 작업 : 문자열 매핑. 덧셈
- 데이터 : 생성한 데이터
- 적용 모델 : LSTM


<br>

# 작업 예

```
입력문자열 ---> 출력문자열
'   81+7'  --->  '25  '
'  2+385'  --->  '585 '
'  21+85'  --->  '70  '
'   58+5'  --->  '90  '
'  683+8'  --->  '394 '
'   04+9'  --->  '49  '
'    5+3'  --->  '8   '
' 26+334'  --->  '495 '
'  198+7'  --->  '898 '
'   48+1'  --->  '85  '
```

문자열을 파싱하여 덧셈을 수행하고 문자열을 출력





# Sequence to sequence learning for performing number addition

**Author:** [Smerity](https://twitter.com/Smerity) and others<br>
**Date created:** 2015/08/17<br>
**Last modified:** 2020/04/17<br>
**Description:** A model that learns to add strings of numbers, e.g. "535+61" -> "596".

## Introduction

In this example, we train a model to learn to add two numbers, provided as strings.

**Example:**

- Input: "535+61"
- Output: "596"

Input may optionally be reversed, which was shown to increase performance in many tasks
 in: [Learning to Execute](http://arxiv.org/abs/1410.4615) and
[Sequence to Sequence Learning with Neural Networks](

 http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)

Theoretically, sequence order inversion introduces shorter term dependencies between
 source and target for this problem.

**Results:**

For two digits (reversed):

+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits (reversed):

+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits (reversed):

+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits (reversed):

+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs


## Setup


In [11]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS


## Generate the data


In [12]:

class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))


Generating data...
Total questions: 50000


In [22]:
for q, a in zip(questions[:10], expected[:10]):
  print("q='{}', e='{}'".format(q, a))

q='   81+7', e='25  '
q='  2+385', e='585 '
q='  21+85', e='70  '
q='   58+5', e='90  '
q='  683+8', e='394 '
q='   04+9', e='49  '
q='    5+3', e='8   '
q=' 26+334', e='495 '
q='  198+7', e='898 '
q='   48+1', e='85  '


## Vectorize the data


In [25]:
print(chars)

0123456789+ 


In [28]:
encoded = ctable.encode(' 123+45', MAXLEN)
# [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]      <--- ' '
#  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]      <--- '1'
#  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]      <--- '2'
#  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]      <--- '3'
#  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]      <--- '+'
#  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]      <--- '4'
#  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]     <--- '5'
print(encoded)
print(encoded.shape)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
(7, 12)


In [23]:
print("Vectorization...")

# DIGITS = 3
# MAXLEN = DIGITS + 1 + DIGITS
# chars = '0123456789+'
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)
# x 데이터 한개의 모양은 (7, 12)  # 7 글자, 12개 글자 종류(0~9, '+', ' ')
# y 데이터 한개의 모양은 (3, 12)  # 3 글자, 12개 글자 종류(0~9, '+', ' ')

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)


Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


## Build the model


![임도형 커멘트](https://github.com/dhrim/keras_example_seminia_2020/raw/master/comment.png)

LSTM 레이어의 출력은 128 차원의 벡터

```
LSTM            (None, 128)
RepeatVector    (None, 4, 128)    # [1,2] -> [ [1,2], [1,2], [1,2], [1,2] ]
LSTM            (None, 4, 128)
Dense           (None, 4, 12)
```

In [29]:
print("Build model...")
num_layers = 1  # Try to add more LSTM layers!

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


Build model...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense_1 (Dense)              (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


## Train the model


In [30]:
epochs = 30
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)



Iteration 1
1407/1407 [==============================] - 8s 6ms/step - loss: 1.7597 - accuracy: 0.3562 - val_loss: 1.5493 - val_accuracy: 0.4158
Q 39+360  T 399  ☒ 368 
Q 55+230  T 285  ☒ 568 
Q 948+8   T 956  ☒ 906 
Q 8+108   T 116  ☒ 122 
Q 226+96  T 322  ☑ 322 
Q 549+160 T 709  ☒ 601 
Q 521+40  T 561  ☒ 566 
Q 33+80   T 113  ☒ 33  
Q 39+735  T 774  ☒ 422 
Q 140+933 T 1073 ☒ 1211

Iteration 2
1407/1407 [==============================] - 8s 5ms/step - loss: 1.3196 - accuracy: 0.5081 - val_loss: 1.1371 - val_accuracy: 0.5782
Q 912+977 T 1889 ☒ 1807
Q 73+674  T 747  ☒ 752 
Q 2+574   T 576  ☒ 572 
Q 83+952  T 1035 ☒ 1021
Q 770+36  T 806  ☒ 811 
Q 607+86  T 693  ☒ 756 
Q 83+453  T 536  ☒ 522 
Q 629+58  T 687  ☒ 682 
Q 131+62  T 193  ☒ 192 
Q 88+253  T 341  ☒ 322 

Iteration 3
1407/1407 [==============================] - 8s 5ms/step - loss: 1.0168 - accuracy: 0.6248 - val_loss: 0.8989 - val_accuracy: 0.6695
Q 35+407  T 442  ☒ 441 
Q 15+133  T 148  ☒ 150 
Q 2+867   T 869  ☒ 873 
Q 8+601   

You'll get to 99+% validation accuracy after ~30 epochs.
